# Collect Environmental Data

I will be using [NASA POWER Project](https://power.larc.nasa.gov/) API to collect environmental data. Before I can collect the environmental data, I need to first find the center coordinates for each ZCTA I will be working with. I am using [this](https://github.com/OpenDataDE/State-zip-code-GeoJSON) GitHub repo which had the exact GeoJSON data I was looking for. I have cloned this repo and moved all of the relevant json files to `data/geography` of this repo. 

ZCTAs are essentially the Census Bureau's definition of the USPS zip codes. In most cases the ZCTA code is the same as the Zip Code for a given area, but there are exceptions. Refer to the Census Bureau's [guide](https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html) on ZCTAs for more info.

The Tracking the Sun report only covers about 28 states (less once I filter the data) and Washington DC, so I will only need the ZCTAs from those states. 

In [1]:
import json
import pandas as pd
import numpy as np

### Collect ZCTA (zipcode) Geographic Data

Below is a list of states that are covered by the Tracking the Sun dataset. These are the only states I will need to collect geographic data for at the moment.

**NOTE**: I renamed the file names from the GitHub repo to follow the format `<state_name>.json`

In [2]:
STATES = ['arizona', 'new_jersey', 'connecticut', 'texas', 'washington', 'ohio', 'florida', 'new_york',
            'oregon', 'illinois', 'district_of_columbia', 'massachusetts', 'new_hampshire', 'california',
            'new_mexico', 'pennsylvania', 'vermont', 'delaware', 'utah', 'wisconsin']

#### Test with California

Before I perform the process for all the state files, I want to work through one to figure out what needs to be done. Below I read in the geojson file for all ZCTAs in California.

In [4]:
with open('../data/geography/california.json') as f:
    ca_codes = json.load(f)

In [5]:
ca_codes.keys()

dict_keys(['type', 'features'])

In [6]:
ca_codes['type']

'FeatureCollection'

In [7]:
type(ca_codes['features'])

list

In [8]:
ca_codes['features'][0]

{'type': 'Feature',
 'properties': {'STATEFP10': '06',
  'ZCTA5CE10': '94601',
  'GEOID10': '0694601',
  'CLASSFP10': 'B5',
  'MTFCC10': 'G6350',
  'FUNCSTAT10': 'S',
  'ALAND10': 8410939,
  'AWATER10': 310703,
  'INTPTLAT10': '+37.7755447',
  'INTPTLON10': '-122.2187049',
  'PARTFLG10': 'N'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.227171, 37.791969],
    [-122.226933, 37.792244],
    [-122.226219, 37.793071],
    [-122.226048, 37.793271],
    [-122.225991, 37.793354],
    [-122.226094, 37.793406],
    [-122.226121, 37.793417],
    [-122.226258, 37.793473],
    [-122.226378, 37.793498],
    [-122.226473, 37.793512],
    [-122.225865, 37.794702],
    [-122.224834, 37.794821],
    [-122.223757, 37.794912],
    [-122.223531, 37.795772],
    [-122.223253, 37.795825],
    [-122.222699, 37.796062],
    [-122.222543, 37.796142],
    [-122.222592, 37.795971],
    [-122.22274, 37.795459],
    [-122.22279, 37.795289],
    [-122.222699, 37.795299],
    [-122.222649, 37.795293],

The `features` key what contains the actual geographic data. It is a list of each ZCTA in the given state

In [9]:
ca_codes['features'][0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [10]:
ca_codes['features'][0]['properties']

{'STATEFP10': '06',
 'ZCTA5CE10': '94601',
 'GEOID10': '0694601',
 'CLASSFP10': 'B5',
 'MTFCC10': 'G6350',
 'FUNCSTAT10': 'S',
 'ALAND10': 8410939,
 'AWATER10': 310703,
 'INTPTLAT10': '+37.7755447',
 'INTPTLON10': '-122.2187049',
 'PARTFLG10': 'N'}

The `properties` key within each Feature element contains information on the corresponding geography. `STATEFP10` is the respective state code and `ZCTA5CE10` is the respective ZCTA for the given geography. I want the coordinates at the center of each ZCTA to collect environmental data for the given geographic location. The center coordinates are `INTPTLAT10` and `INTPTLON10`

Below, I collect the center coordinates for each ZCTA in California. I store these coordinates in a dictionary, `ca_coordinates`, where the key is the ZCTA and the value is a tuple in the following format (lat, lon)

The values in the geojson are all strings. I will convert the ZCTA to an integer and the latitude and longitude to floats.

In [11]:
float(ca_codes['features'][1]['properties']['INTPTLAT10'])

37.7737968

In [12]:
ca_coordinates = {}
for geo in ca_codes['features']:

    # Properties of zcta
    zcta_props = geo['properties']

    # ZCTA (as integer)
    zcta = int(zcta_props['ZCTA5CE10'])

    # Latitude
    lat = float(zcta_props['INTPTLAT10'])

    # Longitude
    lon = float(zcta_props['INTPTLON10'])

    # Store in dictionary
    ca_coordinates[zcta] = (lat, lon)

In [13]:
len(ca_coordinates)

1769

1,769 ZCTAs in California. I can repeat this process for the rest of the states. Below I define a function to perform the above process to extract the center coordinates for each ZCTA in a state.

In [14]:
def get_center_coordinates(codes):

    center_coordinates = {}

    for geo in codes['features']:

        # Properties of ZCTA
        zcta_props = geo['properties']

            # ZCTA (as integer)
        zcta = int(zcta_props['ZCTA5CE10'])

        # Latitude
        lat = float(zcta_props['INTPTLAT10'])

        # Longitude
        lon = float(zcta_props['INTPTLON10'])

        # Store in dictionary
        center_coordinates[zcta] = (lat, lon)

    return center_coordinates

Below I store the contents of each json file in a dictionary `state_zcta`, where the key is the state name and value is the contents of the .json

In [16]:
state_zcta = {}
for state in STATES:

    with open('../data/geography/{}.json'.format(state)) as f:
        geo = json.load(f)

    state_zcta[state] = geo

Now that I have the geojson for the ZCTAs in each state, I can extract the center coordinates for each ZCTA. Using the function defined above, `get_center_coordinates`, I create a dictionary, `state_coordinates` where the key is the state name and the value is a dictionary containing the ZCTA and corresponding central coordinates for each ZCTA in the state.

In [17]:
state_coordinates = {}

for state in state_zcta:
    state_coordinates[state] = get_center_coordinates(state_zcta[state])

In [18]:
state_coordinates['illinois'][60046]

(42.4163231, -88.0591192)

`state_coordinates` has center coordinates for each ZCTA organized by the state they fall within. What I want is a dictionary with the key as each individual ZCTA and the value as tuple of center coordinates.

In [19]:
coordinates = {}

for state in state_coordinates:
    for zcta in state_coordinates[state]:
        coordinates[zcta] = state_coordinates[state][zcta]

In [20]:
len(coordinates)

15755

In [21]:
coordinates

{86506: (35.3580528, -109.2204013),
 86520: (36.1099126, -109.9356124),
 86556: (36.2883628, -109.2554991),
 86545: (36.7114129, -109.606619),
 85638: (31.7161717, -110.0516039),
 85602: (32.1763766, -110.4050931),
 85627: (32.0077865, -110.2355411),
 86320: (35.3461295, -112.6454838),
 86003: (35.1897078, -111.2240145),
 85541: (34.1949986, -111.3050686),
 85531: (32.8841086, -109.790074),
 85086: (33.815971, -112.1198362),
 85382: (33.6542905, -112.2494735),
 85027: (33.6797612, -112.0925291),
 85048: (33.3125521, -112.0571848),
 85335: (33.5922832, -112.3276915),
 85210: (33.3897328, -111.8435049),
 85051: (33.5586588, -112.132418),
 85013: (33.5110869, -112.084749),
 85205: (33.4324202, -111.7185116),
 85351: (33.6061308, -112.2841138),
 85309: (33.5308785, -112.382786),
 85342: (33.9445495, -112.4513742),
 86441: (35.571314, -114.3723321),
 86433: (34.9175644, -114.3488086),
 86438: (34.5489252, -113.8054215),
 86021: (36.9722793, -113.0243182),
 85929: (34.1846834, -109.9398224),

## Collect Environmental Data

I will be using [NASA POWER Project](https://power.larc.nasa.gov/) API to collect environmental data. I have the central coordinates of nearly 16,000 Census ZCTAs, these will be the coordinates I use to call the API. Using the Deep Solar Analysis as a reference, they collected the following environmental factors for each location:
- Air Temperature
- Relative Humidity
- Solar Radiation
- Atmospheric Pressure
- Wind Speed
- Earth Temperature
- Elevation
- Earth Temperature Amplitude
- Number of Frost Days
- Heating Degree Day
- Cooling Degree Day

I will collect the same variables for my analysis, possibly a few more. I intend to focus my analysis on residential PV systems installed 2010-present, I want the average value for each of these variables over that time period.

The API Parameters associated with the variables are as follows:
- Air Temperature (Temperature at 2 Meters) -> T2M
- Relative Humidity (Relative Humidity at 2 Meters) -> RH2M
- Solar Radiation -> SI_EF_TILTED_SURFACE
- Atmospheric Pressure (Surface Pressure) -> PS
- Cloud Amount at Daytime-> CLOUD_AMT_DAY -> Average percent of cloud amount during daylight during temporal period
- Cooling Degree Day (Cooling Degree Days Above 18.3C) -> CDD18_3
- Heating Degree Day (Heating Degree Days Below 18.3C) -> HDD18_3
- Wind Speed (Wind Speed at 2 Meters) -> WS2M
- Earth Temperature (Earth Skin Temperature) -> TS
- Frost Days -> FROST_DAYS
- Earth Temperature Amplitude (Earth Skin Temperature Amplitude) -> TS_AMP
- Solar Azimuth Angle -> SG_SAA (angle of the vector from the earth point to the suns position)

The solar radiation parameter returns multiple values, the one I intend to focus on is SI_EF_TILTED_SURFACE_HORIZONTAL

The API documentation says that solar parameters are available at 1x1 degree resolution, my coordinates are much more granular than this. I will need to round the coordinates for all the ZCTAs to the nearest degree and collect environmental data for the rounded coordinates.

In [22]:
import requests

In [23]:
# List of API Parameters
params = ['T2M', 'RH2M', 'SI_EF_TILTED_SURFACE_HORIZONTAL', 'PS', 'CLOUD_AMT_DAY', 'CDD18_3',
            'HDD18_3', 'WS2M', 'TS', 'FROST_DAYS', 'TS_AMP', 'SG_SAA']

[This](https://power.larc.nasa.gov/docs/tutorials/service-data-request/api/) tutorial from NASA POWER Project shows how to make multiple requests to the API for multiple points. I will be using the Climatology temporal level, collecting data on the above parameters from 2010-2020. I will begin by making a single API call before iterating over all the coordinates.

Below I create a new dictionary `rounded_coordinates` which is the nearest whole degree coordinates and a list of ZCTA nearest to that location. This is the dictionary I will use to collect environmental data.

In [24]:
rounded_coordinates = {}

for zcta in coordinates:

    lat, lon = coordinates[zcta]

    lat_rounded = round(lat, 0)
    lon_rounded = round(lon, 0)

    # Get list of ZCTAs associated with given (lat_rounded, lon_rounded) key
    # If key does not yet exist return empty list
    zcta_list = rounded_coordinates.get((lat_rounded, lon_rounded), [])

    # Append ZCTA to list
    zcta_list.append(zcta)

    # Set list equal to (lat_rounded, lon_rounded) key in dictionary
    rounded_coordinates[(lat_rounded, lon_rounded)] = zcta_list

In [25]:
len(rounded_coordinates)

447

I will only need to make 447 API calls now rather than almost 16,000. I want to make a test API call before collecting data for all coordinates

In [26]:
base_url = "https://power.larc.nasa.gov/api/temporal/climatology/point?parameters=T2M,RH2M,SI_EF_TILTED_SURFACE,PS,CLOUD_AMT_DAY,CDD18_3,HDD18_3,WS2M,TS,FROST_DAYS,TS_AMP,SG_SAA&community=RE&latitude={}&longitude={}&start=2010&end=2020&format=JSON"

In [27]:
test_url = base_url.format(coordinates[60046][0], coordinates[60046][1])
test_url

'https://power.larc.nasa.gov/api/temporal/climatology/point?parameters=T2M,RH2M,SI_EF_TILTED_SURFACE,PS,CLOUD_AMT_DAY,CDD18_3,HDD18_3,WS2M,TS,FROST_DAYS,TS_AMP,SG_SAA&community=RE&latitude=42.4163231&longitude=-88.0591192&start=2010&end=2020&format=JSON'

In [28]:
resp = requests.get(url=test_url, verify=True, timeout=30.00)

In [29]:
resp.status_code

200

In [30]:
data = json.loads(resp.content.decode('utf-8'))
data

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-88.0591192, 42.4163231, 238.39]},
 'properties': {'parameter': {'PS': {'JAN': 98.91,
    'FEB': 98.89,
    'MAR': 98.94,
    'APR': 98.59,
    'MAY': 98.64,
    'JUN': 98.53,
    'JUL': 98.7,
    'AUG': 98.7,
    'SEP': 98.86,
    'OCT': 98.73,
    'NOV': 98.94,
    'DEC': 98.89,
    'ANN': 98.78},
   'TS': {'JAN': -6.47,
    'FEB': -4.93,
    'MAR': 2.03,
    'APR': 7.39,
    'MAY': 14.12,
    'JUN': 19.67,
    'JUL': 23.19,
    'AUG': 22.23,
    'SEP': 18.21,
    'OCT': 10.67,
    'NOV': 2.87,
    'DEC': -2.64,
    'ANN': 8.93},
   'T2M': {'JAN': -6.49,
    'FEB': -4.92,
    'MAR': 1.82,
    'APR': 7.18,
    'MAY': 14.04,
    'JUN': 19.59,
    'JUL': 23.0,
    'AUG': 22.01,
    'SEP': 18.0,
    'OCT': 10.63,
    'NOV': 2.9,
    'DEC': -2.6,
    'ANN': 8.83},
   'RH2M': {'JAN': 88.44,
    'FEB': 85.88,
    'MAR': 79.96,
    'APR': 77.81,
    'MAY': 78.68,
    'JUN': 78.4,
    'JUL': 77.01,
    'AUG': 75.66,
    'SEP

In [31]:
data['properties']['parameter'].keys()

dict_keys(['PS', 'TS', 'T2M', 'RH2M', 'WS2M', 'SG_SAA', 'TS_AMP', 'CDD18_3', 'HDD18_3', 'FROST_DAYS', 'CLOUD_AMT_DAY', 'SI_EF_TILTED_SURFACE_HORIZONTAL', 'SI_EF_TILTED_SURFACE_LAT_MINUS15', 'SI_EF_TILTED_SURFACE_LATITUDE', 'SI_EF_TILTED_SURFACE_LAT_PLUS15', 'SI_EF_TILTED_SURFACE_VERTICAL', 'SI_EF_TILTED_SURFACE_OPTIMAL', 'SI_EF_TILTED_SURFACE_OPTIMAL_ANG', 'SI_EF_TILTED_SURFACE_OPTIMAL_ANG_ORT'])

There are a few variables I do not need. `params` list contains all of the parameters I am interested in.

The API call returned the average value for each parameter for each month. I need to find the true average by finding the mean value across the year. The key `ANN` will need to be ignored.

In [32]:
round(np.mean(list(data['properties']['parameter'][params[0]].values())[:-1]), 2)

8.76

In [33]:
test_zipcode = {}
for param in params:

    # Values for each month of given parameter
    vals = list(data['properties']['parameter'][param].values())[:-1]

    test_zipcode[param] = round(np.mean(vals), 2)

In [34]:
test_zipcode

{'T2M': 8.76,
 'RH2M': 80.42,
 'SI_EF_TILTED_SURFACE_HORIZONTAL': 3.88,
 'PS': 98.78,
 'CLOUD_AMT_DAY': 61.03,
 'CDD18_3': 35.04,
 'HDD18_3': 310.22,
 'WS2M': 3.43,
 'TS': 8.86,
 'FROST_DAYS': 11.13,
 'TS_AMP': 14.08,
 'SG_SAA': -89.04}

Now I want to repeat this process for all of the ZCTAs. The final result should be a dictionary of dictionaries. Where the keys of the outer dictionary is the rounded coordinates and the value is all of the parameter averages for that location. I am going to define a few functions to break up the process.

`call_nasa_api`:
- accepts url and latitude and longitude
- makes call to NASA power API
- returns the decoded response

`compute_averages`:
- accepts the data and list of parameters
- computers average for each parameter in `data`
- returns dictionary of averages for each environmental variable of interest

In [35]:
def call_nasa_api(_url, lat, lon):

    api_url = _url.format(lat, lon)

    resp = requests.get(url=api_url, verify=True, timeout=60.00)

    data = json.loads(resp.content.decode('utf-8'))

    return data

In [36]:
def compute_averages(data, parameters):

    d = {}

    for param in parameters:
        
        vals = list(data['properties']['parameter'][param].values())[:-1]

        d[param] = round(np.mean(vals), 2)
    
    return d

Now I want to test the functions on one more ZCTA before doing all of them

In [37]:
coordinates[78125]

(28.5408999, -97.9570008)

In [38]:
test_call = call_nasa_api(base_url, coordinates[78125][0], coordinates[78125][1])

In [39]:
test_avgs = compute_averages(test_call, params)

In [40]:
test_avgs

{'T2M': 22.3,
 'RH2M': 69.38,
 'SI_EF_TILTED_SURFACE_HORIZONTAL': 4.82,
 'PS': 100.49,
 'CLOUD_AMT_DAY': 49.36,
 'CDD18_3': 190.74,
 'HDD18_3': 42.83,
 'WS2M': 3.01,
 'TS': 23.08,
 'FROST_DAYS': 0.43,
 'TS_AMP': 15.35,
 'SG_SAA': -93.96}

Finally I can iterate over all of the coordinate pairs in `rounded_coordinates` and get the average value for each parameter. One thing I need to be aware of is the API's Rate Limiting. It only allows 60 API calls per minute to the endpoint I am using.

In [44]:
avg_parameters = {}

for coords in rounded_coordinates:

    lat, lon = coords[0], coords[1]

    # Call API
    api_resp = call_nasa_api(base_url, lat, lon)

    # Compute averages for each parameter
    avgs = compute_averages(api_resp, params)

    # Add averages to dictionary
    avg_parameters[coords] = avgs

In [45]:
len(avg_parameters)

447

That all seemed to work. Now I have the average environmental conditions for the 533 coordinate pairs. Next I want to create a dictionary where each key is a ZCTA and the value is corresponding climate conditions. 

I will iterate through `avg_parameters` and collect the list of ZCTAs belonging to each coordinate pair from `rounded_coordinates`. Then add each ZCTA as it's own key with value equal to same climate dictionary as the rounded coordinates key I extracted the ZCTA from.

In [46]:
zcta_env = {}

for coord in avg_parameters:
    
    # environmental data
    env = avg_parameters[coord]

    # List of ZCTAs
    zctas = rounded_coordinates[coord]

    # Iterate over list of ZCTAs
    for z in zctas:

        # Set as key-value pair in dictionary with associated environmental data
        zcta_env[z] = env

In [47]:
len(zcta_env)

15755

In [48]:
zcta_env

{86506: {'T2M': 11.15,
  'RH2M': 45.83,
  'SI_EF_TILTED_SURFACE_HORIZONTAL': 5.49,
  'PS': 79.0,
  'CLOUD_AMT_DAY': 40.9,
  'CDD18_3': 34.21,
  'HDD18_3': 228.16,
  'WS2M': 2.93,
  'TS': 11.94,
  'FROST_DAYS': 10.64,
  'TS_AMP': 18.21,
  'SG_SAA': -100.91},
 86512: {'T2M': 11.15,
  'RH2M': 45.83,
  'SI_EF_TILTED_SURFACE_HORIZONTAL': 5.49,
  'PS': 79.0,
  'CLOUD_AMT_DAY': 40.9,
  'CDD18_3': 34.21,
  'HDD18_3': 228.16,
  'WS2M': 2.93,
  'TS': 11.94,
  'FROST_DAYS': 10.64,
  'TS_AMP': 18.21,
  'SG_SAA': -100.91},
 85936: {'T2M': 11.15,
  'RH2M': 45.83,
  'SI_EF_TILTED_SURFACE_HORIZONTAL': 5.49,
  'PS': 79.0,
  'CLOUD_AMT_DAY': 40.9,
  'CDD18_3': 34.21,
  'HDD18_3': 228.16,
  'WS2M': 2.93,
  'TS': 11.94,
  'FROST_DAYS': 10.64,
  'TS_AMP': 18.21,
  'SG_SAA': -100.91},
 86508: {'T2M': 11.15,
  'RH2M': 45.83,
  'SI_EF_TILTED_SURFACE_HORIZONTAL': 5.49,
  'PS': 79.0,
  'CLOUD_AMT_DAY': 40.9,
  'CDD18_3': 34.21,
  'HDD18_3': 228.16,
  'WS2M': 2.93,
  'TS': 11.94,
  'FROST_DAYS': 10.64,
  'TS_AMP

Below I save this dictionary as a new json file `zcta_env.json` in data/

In [50]:
with open('../data/zcta_env.json', 'w') as f:
    json.dump(zcta_env, f)